# Clickhouse

This section provides an overview of features related to intefacing with the Clickhouse database. The [`clickhouse_connect`](https://pypi.org/project/clickhouse-connect/) is a database driver for python. Check the the description at the [clickhouse site](https://clickhouse.com/docs/integrations/python).

In [4]:
import clickhouse_connect
from src.rerun_docker import reload_docker_container

container = reload_docker_container(
    image="clickhouse/clickhouse-server:24",
    name="clickhouse_examples",
    detach=True,
    remove=True,
    ports={
        "8123": 8123,
    },
)

## Executing

To execute the command, you must create a client using the `clickhouse_connect.get_client` method and use the `query` method of the gotten object.

---

The following cell shows the process of creating a connection.

In [5]:
client = clickhouse_connect.get_client(
    host="localhost",
    port=8123,
    username="default",
    password="",
    database="default",
)

The following code shows the structure of the output.

In [6]:
query_result = client.query("SELECT 10 v10, 20 v20, 30 v30, 40 v40;")
print("Column names:", query_result.column_names)
print("Data:", query_result.result_rows)

Column names: ('v10', 'v20', 'v30', 'v40')
Data: [(10, 20, 30, 40)]


Next cell executes some command that shouldn't have any output.

In [7]:
query_result = client.query("""
    CREATE TABLE
    IF NOT EXISTS 
    test_table (v10 UInt8, v20 UInt8) 
    ENGINE = MergeTree ORDER BY v10;
""")

print("Column names:", query_result.column_names)
print("Data:", query_result.result_rows)

Column names: ('read_rows', 'read_bytes', 'written_rows', 'written_bytes', 'total_rows_to_read', 'result_rows', 'result_bytes', 'elapsed_ns', 'query_id')
Data: [[0, 0, 0, 0, 0, 0, 0, 9927398, 'e693da30-7d1d-4e0e-964f-8e788525b16f']]


But it looks like it has some technical information.

In [ ]:
client.close()

## Multistatement

`clickhouse_connect` doesn't allow to run multistatement code.

---

The following cell shows the type of the error you would get in attempt to execute `multistatement` sql code using `clickhouse_connect`.

In [12]:
try:
    client.query("SELECT 1 as val1; SELECT 2 as val2;")
except Exception as e:
    print(e)

HTTPDriver for http://localhost:8123 received ClickHouse error code 62
 Code: 62. DB::Exception: Syntax error (Multi-statements are not allowed): failed at position 17 (end of query) (line 1, col 17): ; SELECT 2 as val2
 FORMAT Native. . (SYNTAX_ERROR) (version 24.10.2.80 (official build))

